## NOTEBOOK 01-Primeros pasos. Ingest and validate

In [2]:
from pathlib import Path
ROOT = Path("/home/jovyan/inesagent")
assert ROOT.exists()
print("ROOT:", ROOT)



ROOT: /home/jovyan/inesagent


In [3]:
#definimos rutas
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")  # tu raíz real
PATH_UNANNOT = ROOT / "data" / "corpus_unannotated.jsonl"
PATH_GOLD    = ROOT / "gold" / "corpus_annotated.jsonl"

OUT_SPLITS   = ROOT / "outputs" / "splits"
OUT_MEMORY   = ROOT / "outputs" / "memory"
OUT_PRED     = ROOT / "outputs" / "predictions"
RES_METRICS  = ROOT / "results" / "metrics"

for p in [OUT_SPLITS, OUT_MEMORY, OUT_PRED, RES_METRICS]:
    p.mkdir(parents=True, exist_ok=True)


Hemos visto que en los corpus en JSONL aparece el texto sin decodificar, y tanto tildes como símbolos aparecen con códigos. Ej. p\u00f3liza = póliza. Corregimos esto ya que en las guías (guidelines) los extractos de ejemplos están unificados y los apartaremos haciendo matching

In [4]:
#cargamos JSONL correctamente (para decodificar y unificar)
import json

def load_jsonl(path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for i, line in enumerate(f):
            line = line.strip()
            if not line:
                continue
            rows.append(json.loads(line))  # decodifica \uXXXX automáticamente
    return rows

unannot = load_jsonl(PATH_UNANNOT)
gold = load_jsonl(PATH_GOLD)

len(unannot), len(gold)


(373, 373)

Teniendo en cuenta que los formatos del corpus sin anotar (corpus_unannotated) y anotado/gold (corpus_annotated) son los siguientes: 
Sin anotar: {"text": ...} (sin id)
Gold: {"id": ..., "text": ..., "tags": [...]}
Validamos un esquema mínimo con el que detectaremos si hay problemas de formato y así evitar errores raros más adelante.

In [5]:
def check_unannot(row):
    assert "text" in row and isinstance(row["text"], str)

def check_gold(row):
    assert "id" in row
    assert "text" in row and isinstance(row["text"], str)
    assert "tags" in row and isinstance(row["tags"], list)

for r in unannot[:100]:
    check_unannot(r)

for r in gold[:100]:
    check_gold(r)

print("OK")


OK


Validamos offsets del gold (imprescindible para spans). Si hay offsets rotos, la evaluación y el agente fallarán

In [6]:
def validate_offsets(doc):
    text = doc["text"]
    for t in doc["tags"]:
        start, end = t["start"], t["end"]
        assert 0 <= start < end <= len(text)
        span = text[start:end]
        assert span != ""

bad = 0
for d in gold:
    try:
        validate_offsets(d)
    except Exception:
        bad += 1

bad


0

Generamos estadísticas de etiquetas para después confirmar que nos centraremos en 4 de las más frecuentes y más relevantes: 
    "OBJETO",
    "PRECIO_DEL_CONTRATO",
    "DURACION_TOTAL_DEL_CONTRATO",
    "RESOLUCION",
(si se tiene en cuenta frecuencia, 'GARANTIA_ECONOMICA' estaría incluida, pero preferimos 'PRECIO_DEL_CONTRATO')

Para documentarlo: “El corpus contiene N categorías anotadas… Se analizó la distribución de frecuencias… Para este trabajo se seleccionaron las cuatro categorías más frecuentes o relevantes… El resto de categorías se descartaron para el MVP”

In [7]:
from collections import Counter

label_counts = Counter()
for d in gold:
    for t in d["tags"]:
        label_counts[t["tag"]] += 1

label_counts.most_common(20)


[('OBJETO', 925),
 ('RESOLUCION', 526),
 ('DURACION_TOTAL_DEL_CONTRATO', 468),
 ('GARANTIA_ECONOMICA', 447),
 ('PRECIO_DEL_CONTRATO', 436),
 ('PENALIDAD_POR_DEMORA', 338),
 ('CONFIDENCIALIDAD', 291),
 ('PENALIDAD_POR_CUMPLIMIENTO_DEFECTUOSO', 265),
 ('VERIFICACION_DEL_CUMPLIMIENTO', 263),
 ('PROTECCION_DE_DATOS', 257),
 ('INDEMNIZACION_DE_DAÑOS_Y_PERJUICIOS_POR_INCUMPLIMIENTO', 175),
 ('GARANTIA_PLAZO', 170),
 ('GARANTIA_AVAL_BANCARIO', 78),
 ('INDEMNIZACION_DE_DAÑOS_A_TERCEROS', 67),
 ('GARANTIA_RETENCION_DEL_PRECIO', 40),
 ('GARANTIA_SEGURO_DE_CAUCION', 25),
 ('CANON', 6),
 ('PROPIEDAD_INTELECTUAL', 5)]

Filtramos: indicamos explícitamente el subconjunto MVP (decisión de diseño) de las etiquetas elegidas (definición del scope del sgente)

In [8]:
MVP_LABELS = {
    "OBJETO",
    "PRECIO_DEL_CONTRATO",
    "DURACION_TOTAL_DEL_CONTRATO",
    "RESOLUCION",
}


mvp_counts = {
    label: label_counts[label]
    for label in MVP_LABELS
}

mvp_counts


{'DURACION_TOTAL_DEL_CONTRATO': 468,
 'RESOLUCION': 526,
 'PRECIO_DEL_CONTRATO': 436,
 'OBJETO': 925}